# Reading Movies

In [58]:
import ast
import csv
movies = []
with open('Output.csv') as f:
    reader = csv.reader(f)
    first_read = True
    list_of_column_names = []
    
    for item in reader:
        if first_read:
            list_of_column_names.extend(item)
            first_read = False
            continue
            
        movie = {}
        for value, column_name in zip(item, list_of_column_names):
            movie[column_name] = value
            
        movie["query_results"] = ast.literal_eval(movie["query_results"])
        movie["top_10_results"] = movie["query_results"][:10]
        movies.append(movie)
        #print(movie)

exclude_channels = []
with open("Hand_Excluded_Channels.csv") as f:
  reader = csv.reader(f)
  lst = reader
  for item in lst:
    exclude_channels.append(item[0])

with open("Programmatically_Excluded_Channels.csv") as f:
  reader = csv.reader(f)
  lst = reader
  for item in lst:
    exclude_channels.append(item[0])

known_channels = []
with open("Hand_Known_Channels.csv") as f:
  reader = csv.reader(f)
  lst = reader
  for item in lst:
    known_channels.append(item[0])




In [59]:
len(movies)

1967

In [60]:
import re
def translate(to_translate):
		tabin = u'áéíóú'
		tabout = u'aeiou'
		tabin = [ord(char) for char in tabin]
		translate_table = dict(zip(tabin, tabout))
		return to_translate.translate(translate_table)

def remove_non_alphanumeric(result):
  return re.sub(r'[^a-zA-Z0-9]', '', result)

def normalize(token):
		result = token.lower()
		result = translate(result)
		result = remove_non_alphanumeric(result)
		return result

In [61]:
from datetime import date

def diff_dates(date1, date2):
  return abs(date2 - date1).days


def anterior(date1, date2):
  return (date2 - date1).days >= 0

def remove_results_by_date(release_start, query_results):
        anterior_query_results = []
        for query_result in query_results:
            title, channel, video_id, result_date_str, premiered, stream, subscribers, views, likes = query_result

            if release_start == None or result_date_str == None or release_start == "" or result_date_str == "":
                is_anterior = True
            else:
                #print("{} {}".format(repr(release_start), result_date_str))
                movie_moth, movie_day, movie_year = release_start.split("/")
                result_moth, result_day, result_year = result_date_str.split("/")
                movie_date = date(int(movie_year), int(movie_moth), int(movie_day))
                result_date = date(int(result_year), int(result_moth), int(result_day))
                is_anterior = anterior(result_date, movie_date)

            #self.logger.debug("Movie Date {},  Result Date {}, Diff {} Result Date Is Anterior? {}".format(release_start, result_date_str, diff_dates(movie_date, result_date), is_anterior))

            if is_anterior:
                anterior_query_results.append(query_result)
        return anterior_query_results

In [62]:
def remove_results_by_channel(distributor, ampliated_top_k):
  results = []
  for result in ampliated_top_k:
    if result[1].lower() in known_channels:
      results.append(result)

  #if distributor == "Universal Pictures Distributor": ##To Do
    #return get_results_from_channel()

  return results

def exclude_by_channel(ampliated_top_k):
  results = []
  for result in ampliated_top_k:
    if result[1].lower() not in exclude_channels:
      results.append(result)
  return results
  

In [63]:
def score(title_tokens, channel_tokens, query_tokens):
  normalized_title_tokens = []
  for title_token in title_tokens:
    normalized_title_token = normalize(title_token)
    if len(normalized_title_token) > 1:
      normalized_title_tokens.append(normalized_title_token)

  normalized_channel_tokens = []
  for channel_token in channel_tokens:
    normalized_channel_tokens.append(normalize(channel_token))

  normalized_query_tokens = []
  for query_token in query_tokens:
    normalized_query_token = normalize(query_token)
    if len(normalized_query_token) > 1:
      normalized_query_tokens.append(normalized_query_token)
      
  matches = 0
  for normalized_query_token in normalized_query_tokens:
    if normalized_query_token in normalized_title_tokens:
      matches += 1
  score = matches / len(normalized_query_tokens)
  return score

In [64]:
  """for movie in movies:
    scored_trailers = []

    channel_excluded_results = exclude_by_channel(movie["top_10_results"])

    movie["anterior_channel_excluded_results"] = remove_results_by_date(movie["release_start"], channel_excluded_results)

    movie["known_channel_results"] = remove_results_by_channel(movie["distributor"], movie["anterior_channel_excluded_results"]) #OPTION 1

    results = movie["known_channel_results"]

    if results == []:
      results = movie["anterior_channel_excluded_results"]

    for trailer_primary_result in results:
      title, channel, video_id, video_date, premiered, stream, subscribers, views, likes = trailer_primary_result

      if premiered or stream:
        continue

      title_tokens = title.split(" ")
      channel_tokens = channel.split(" ")
      query_tokens = query_tokens = "{} official trailer".format(movie["movie_name"]).split()

      score_result = score(title_tokens, channel_tokens, query_tokens)
      

      scored_trailers.append([title, channel, video_id, video_date, premiered, stream, subscribers, views, likes, score_result])
    movie["scored_trailers"] = scored_trailers"""


'for movie in movies:\n  scored_trailers = []\n\n  channel_excluded_results = exclude_by_channel(movie["top_10_results"])\n\n  movie["anterior_channel_excluded_results"] = remove_results_by_date(movie["release_start"], channel_excluded_results)\n\n  movie["known_channel_results"] = remove_results_by_channel(movie["distributor"], movie["anterior_channel_excluded_results"]) #OPTION 1\n\n  results = movie["known_channel_results"]\n\n  if results == []:\n    results = movie["anterior_channel_excluded_results"]\n\n  for trailer_primary_result in results:\n    title, channel, video_id, video_date, premiered, stream, subscribers, views, likes = trailer_primary_result\n\n    if premiered or stream:\n      continue\n\n    title_tokens = title.split(" ")\n    channel_tokens = channel.split(" ")\n    query_tokens = query_tokens = "{} official trailer".format(movie["movie_name"]).split()\n\n    score_result = score(title_tokens, channel_tokens, query_tokens)\n    \n\n    scored_trailers.append([ti

# Creating manual file


In [65]:
"""with open("score_scheme.csv", "w") as f:
  f.write("movie_name,video_url,video_title,channel,score,id,official_trailer,known_channels_used")
  f.write("\r\n")
  for movie in movies:
    if movie["no_prerelease"] == '0':
      if movie["scored_trailers"] == []:
        movie_name = movie["movie_name"].replace(",","")
        f.write("{},{},{},{},{},{},{},{}".format(movie_name, None, None, None, None, None, None, None))
        f.write("\r\n")

      counter = 1
      for trailer in movie["scored_trailers"]:
        title, channel, video_id, video_date, premiered, stream, subscribers, views, likes, score_result = trailer
        youtube_url = "https://www.youtube.com/watch?v={}".format(video_id)
        movie_name = movie["movie_name"].replace(",","")
        title = title.replace(",","")
        channel = channel.replace(",","")
        official_trailer = 0
        if score_result >= 0.75:
          official_trailer = 1

        score_result = str(round(score_result, 2))
        
        kwnown_channels_used = 0
        if movie["known_channel_results"] != []:
          kwnown_channels_used = 1
        f.write("{},{},{},{},{},{},{},{}".format(movie_name, youtube_url, title, channel, score_result, counter, official_trailer, kwnown_channels_used))
        f.write("\r\n")
        counter += 1
      f.write(",,,,,,,")
      f.write("\r\n")"""

'with open("score_scheme.csv", "w") as f:\n  f.write("movie_name,video_url,video_title,channel,score,id,official_trailer,known_channels_used")\n  f.write("\r\n")\n  for movie in movies:\n    if movie["no_prerelease"] == \'0\':\n      if movie["scored_trailers"] == []:\n        movie_name = movie["movie_name"].replace(",","")\n        f.write("{},{},{},{},{},{},{},{}".format(movie_name, None, None, None, None, None, None, None))\n        f.write("\r\n")\n\n      counter = 1\n      for trailer in movie["scored_trailers"]:\n        title, channel, video_id, video_date, premiered, stream, subscribers, views, likes, score_result = trailer\n        youtube_url = "https://www.youtube.com/watch?v={}".format(video_id)\n        movie_name = movie["movie_name"].replace(",","")\n        title = title.replace(",","")\n        channel = channel.replace(",","")\n        official_trailer = 0\n        if score_result >= 0.75:\n          official_trailer = 1\n\n        score_result = str(round(score_res

# Creating Debug Files

# Top 10 > Filter Date

In [66]:
with open("./Debug_Files/query_results.csv", "w") as f:
    f.write("uniqueID,movie_name,video_url,video_title,channel,score,days_diff,no_prerelease")
    f.write("\r\n")
    for movie in movies:
        query_tokens = "{} official trailer".format(movie["movie_name"]).split()
        for result in movie["query_results"]:
            title, channel, video_id, video_date, premiered, stream, subscribers, views, likes = result

            youtube_url = "https://www.youtube.com/watch?v={}".format(video_id)
            movie_name = movie["movie_name"].replace(",", "")
            title = title.replace(",", "")
            channel = channel.replace(",", "")

            channel_tokens = channel.split(" ")
            title_tokens = title.split(" ")
            score_result = score(channel_tokens=channel_tokens, query_tokens=query_tokens, title_tokens=title_tokens)

            try:
                movie_moth, movie_day, movie_year = movie["release_start"].split("/")
                result_moth, result_day, result_year = video_date.split("/")
                movie_date = date(int(movie_year), int(movie_moth), int(movie_day))
                result_date = date(int(result_year), int(result_moth), int(result_day))
                days_diff_video_movie = (result_date - movie_date).days
            except Exception as e:
                days_diff_video_movie = 1

            f.write("{},{},{},{},{},{},{},{}".format(movie["uniqueID"], movie_name, youtube_url, title, channel,
                                                        score_result, days_diff_video_movie, movie["no_prerelease"]))
            f.write("\r\n")

        f.write(",,,,,,,,")
        f.write("\r\n")

In [67]:
with open("./Debug_Files/date_filtered.csv", "w") as f:
    f.write("uniqueID,movie_name,video_url,video_title,channel,score,days_diff")
    f.write("\r\n")
    date_movies = []
    for movie in movies:
        if movie["no_prerelease"] == "1":
            continue
        results = []

        query_tokens = "{} official trailer".format(movie["movie_name"]).split()

        for result in movie["query_results"]:
            title, channel, video_id, video_date, premiered, stream, subscribers, views, likes = result

            try:
                movie_moth, movie_day, movie_year = movie["release_start"].split("/")
                result_moth, result_day, result_year = video_date.split("/")
                movie_date = date(int(movie_year), int(movie_moth), int(movie_day))
                result_date = date(int(result_year), int(result_moth), int(result_day))
                days_diff_video_movie = (result_date - movie_date).days
            except Exception as e:
                days_diff_video_movie = 1

            if days_diff_video_movie >= 0:
                continue

            youtube_url = "https://www.youtube.com/watch?v={}".format(video_id)
            movie_name = movie["movie_name"].replace(",", "")
            title = title.replace(",", "")
            channel = channel.replace(",", "")

            channel_tokens = channel.split(" ")
            title_tokens = title.split(" ")
            score_result = score(channel_tokens=channel_tokens, query_tokens=query_tokens, title_tokens=title_tokens)

            f.write("{},{},{},{},{},{},{}".format(movie["uniqueID"], movie_name, youtube_url, title, channel, score_result, days_diff_video_movie))
            f.write("\r\n")
            results.append([youtube_url, title, channel, score_result, days_diff_video_movie])

        movie["date_filtered_results"] = results
        date_movies.append(movie)
        f.write(",,,,,,,")
        f.write("\r\n")

In [68]:
with open("./Debug_Files/date_filtered_top_10.csv", "w") as f:
    f.write("uniqueID,movie_name,video_url,video_title,channel,score,days_diff")
    f.write("\r\n")
    top_movies = []
    for movie in date_movies:
        movie_name = movie["movie_name"].replace(",", "")

        for result in movie["date_filtered_results"][:10]:
            youtube_url, title, channel, score_result, days_diff_video_movie = result

            f.write("{},{},{},{},{},{},{}".format(movie["uniqueID"], movie_name, youtube_url, title, channel, score_result, days_diff_video_movie))
            f.write("\r\n")

        movie["top_10"] = movie["date_filtered_results"][:10]
        top_movies.append(movie)

        f.write(",,,,,,,")
        f.write("\r\n")

In [69]:
with open("./Debug_Files/date_filtered_top_10_channel_excluded.csv", "w") as f:
    f.write("uniqueID,movie_name,video_url,video_title,channel,score,days_diff")
    f.write("\r\n")
    channel_movies = []
    for movie in top_movies:
        results = []
        movie_name = movie["movie_name"].replace(",", "")

        for result in movie["top_10"]:
            youtube_url, title, channel, score_result, days_diff_video_movie = result

            if channel.lower() in exclude_channels:
                continue

            f.write("{},{},{},{},{},{},{}".format(movie["uniqueID"], movie_name, youtube_url, title, channel, score_result, days_diff_video_movie))
            f.write("\r\n")
            results.append([youtube_url, title, channel, score_result, days_diff_video_movie])

        movie["channel_excluded_results"] = results
        channel_movies.append(movie)

        if len(results) == 0:
            movie_name = movie["movie_name"].replace(",","")
            f.write("{},{},{},{},{},{},{}".format(movie["uniqueID"], movie_name, None, None, None, None, None))
            f.write("\r\n")
        f.write(",,,,,,,")
        f.write("\r\n")

Another Creating Debug Files


In [41]:
import csv

with open("./Debug_Files/date_filtered_top_10_channel_excluded.csv", "r") as f:
    channel_excluded = []
    reader = csv.reader(f)
    first_read = True
    list_of_column_names = []

    for item in reader:
        if first_read:
            list_of_column_names.extend(item)
            first_read = False
            continue
        if item[0] == "" or item[2] == 'None':
            continue
        movie = {}
        for value, column_name in zip(item, list_of_column_names):
            movie[column_name] = value
        channel_excluded.append(movie)

In [42]:
uniqueIDS = []
for movie in channel_excluded:
    try:
        if int(movie["days_diff"]) <= -365:
            if movie["uniqueID"] not in uniqueIDS:
                uniqueIDS.append(movie["uniqueID"])
    except Exception as e:
        print(movie)
        #print(movie["uniqueID"])

In [43]:
len(uniqueIDS)

327

In [164]:
uniqueIDS

['3',
 '8',
 '15',
 '19',
 '26',
 '27',
 '35',
 '36',
 '37',
 '41',
 '44',
 '51',
 '55',
 '59',
 '60',
 '61',
 '72',
 '82',
 '84',
 '87',
 '100',
 '102',
 '103',
 '104',
 '105',
 '113',
 '126',
 '128',
 '138',
 '150',
 '155',
 '161',
 '185',
 '191',
 '204',
 '216',
 '227',
 '234',
 '237',
 '247',
 '251',
 '286',
 '295',
 '297',
 '298',
 '304',
 '321',
 '325',
 '326',
 '327',
 '347',
 '363',
 '368',
 '371',
 '373',
 '401',
 '405',
 '434',
 '439',
 '444',
 '448',
 '451',
 '458',
 '459',
 '467',
 '469',
 '473',
 '489',
 '497',
 '498',
 '509',
 '553',
 '555',
 '569',
 '571',
 '578',
 '586',
 '598',
 '601',
 '606',
 '617',
 '623',
 '640',
 '644',
 '647',
 '664',
 '677',
 '683',
 '686',
 '711',
 '716',
 '720',
 '722',
 '726',
 '731',
 '757',
 '758',
 '762',
 '763',
 '772',
 '777',
 '784',
 '790',
 '796',
 '801',
 '808',
 '821',
 '825',
 '837',
 '871',
 '879',
 '891',
 '908',
 '914',
 '922',
 '928',
 '942',
 '956',
 '957',
 '962',
 '970',
 '1.002',
 '1.013',
 '1.016',
 '1.024',
 '1.034',
 '1.